<a href="https://colab.research.google.com/github/Vitor-Sallenave/Formacao-em-NLP/blob/main/ML-and-DL/Spam_Classifier_Embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ***◼️ Libraries***

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Embedding
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from google.colab import files

## ***◼️ Dataset***

In [ ]:
# files.upload()

In [ ]:
spams = pd.read_csv('spam.csv')
spams.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


## ***◼️ Transforming the data***

In [ ]:
# Creating a trasnformer to encode the target values: "Category"
le = LabelEncoder()
encoded_categories = le.fit_transform(spams.Category)

print(le.classes_)
print(encoded_categories)

['ham' 'spam']
[0 0 1 ... 0 0 0]


## ***◼️ Vectorizing***

In [ ]:
# Creating the vectorizer
# It will use the 1000 most frequent words
tk = Tokenizer(num_words=1000)
tk.fit_on_texts(spams.Message)
sequences = tk.texts_to_sequences(spams.Message)

print(len(sequences))
print(sequences)

5572
[[49, 471, 840, 755, 658, 65, 8, 88, 123, 351, 148, 67, 58, 144], [46, 336, 472, 6], [47, 489, 8, 19, 4, 796, 901, 2, 175, 659, 261, 71, 2, 2, 337, 489, 555, 960, 73, 390, 179, 660, 391], [6, 247, 150, 23, 381, 6, 139, 153, 57, 150], [1, 98, 108, 69, 490, 2, 961, 69, 221, 112, 473], [797, 128, 67, 145, 109, 158, 21, 7, 38, 338, 89, 902, 55, 116, 413, 3, 44, 12, 14, 85, 46, 365, 960, 2, 68, 323, 234, 2], [210, 11, 633, 9, 25, 55, 2, 382, 36, 10, 110, 718, 10, 55], [72, 235, 13, 798, 118, 109, 609, 72, 13, 12, 51, 841, 392, 2, 13, 248], [719, 72, 4, 842, 440, 236, 3, 17, 109, 441, 2, 154, 962, 2, 129, 16, 129, 414, 516, 963, 581, 65], [136, 13, 97, 686, 26, 133, 6, 82, 2, 491, 2, 5, 324, 534, 903, 36, 339, 12, 47, 16, 5, 97, 491, 243, 47, 18], [30, 237, 35, 80, 222, 7, 1, 98, 70, 2, 288, 81, 40, 289, 226, 95, 208, 442, 90], [2, 175, 159, 48, 720, 2, 904, 443, 634, 73, 7, 68, 2, 373, 187, 64, 252, 391, 94, 41, 721], [195, 3, 17, 191, 4, 119, 113, 47, 8, 92, 517, 443, 154, 73, 5, 338,

In [ ]:
# Padding the sequences, so that they will have the same size and be able to be
# analyzed by the neural network
sequences_padded = pad_sequences(sequences, padding='post', maxlen=500)

## ***◼️ Creating the Neural Network***

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(sequences_padded,
                                                    encoded_categories,
                                                    test_size=0.3)

In [ ]:
print(X_train.shape)
print(sequences_padded.shape)

(3900, 500)
(5572, 500)


In [ ]:
# Creating the sequential model
model = Sequential()

In [ ]:
vocabulary = len(tk.word_index)
print(vocabulary)

9004


In [ ]:
# Adding layers to the model

# Creating an embedding layer
model.add(Embedding(input_dim=vocabulary, output_dim=50,
                    input_length=500))

# Flatten is used to connect Dense and Embedding Layers
model.add(Flatten())

# Units: number of neurons
model.add(Dense(units=10, activation='relu'))

# Dropout layers serve to minimize overfitting by deleting random sinapses
model.add(Dropout(0.1))

# Last layer
model.add(Dense(1, activation='sigmoid'))

In [ ]:
# Defining the compilation configuration of the model for training
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 500, 50)           450200    
                                                                 
 flatten_1 (Flatten)         (None, 25000)             0         
                                                                 
 dense_11 (Dense)            (None, 10)                250010    
                                                                 
 dropout_7 (Dropout)         (None, 10)                0         
                                                                 
 dense_12 (Dense)            (None, 1)                 11        
                                                                 
Total params: 700221 (2.67 MB)
Trainable params: 700221 (2.67 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# Selecting a specific layer
model.get_layer(index=0)

In [ ]:
# Fitting the model
# Epochs: number of iterations
# Batch_size: The number of registers need to update the network weigths
model.fit(X_train, y_train, epochs=20, batch_size=10,
          verbose=True, validation_data=(X_test, y_test))

Epoch 1/20
390/390 [==============================] - 3s 6ms/step - loss: 0.0683 - accuracy: 0.9244 - val_loss: 0.0367 - val_accuracy: 0.9850
Epoch 2/20
390/390 [==============================] - 2s 5ms/step - loss: 0.0350 - accuracy: 0.9846 - val_loss: 0.0294 - val_accuracy: 0.9874
Epoch 3/20
390/390 [==============================] - 2s 6ms/step - loss: 0.0257 - accuracy: 0.9910 - val_loss: 0.0248 - val_accuracy: 0.9862
Epoch 4/20
390/390 [==============================] - 2s 5ms/step - loss: 0.0197 - accuracy: 0.9928 - val_loss: 0.0210 - val_accuracy: 0.9880
Epoch 5/20
390/390 [==============================] - 2s 5ms/step - loss: 0.0156 - accuracy: 0.9938 - val_loss: 0.0184 - val_accuracy: 0.9886
Epoch 6/20
390/390 [==============================] - 2s 6ms/step - loss: 0.0123 - accuracy: 0.9951 - val_loss: 0.0159 - val_accuracy: 0.9898
Epoch 7/20
390/390 [==============================] - 2s 5ms/step - loss: 0.0103 - accuracy: 0.9949 - val_loss: 0.0246 - val_accuracy: 0.9749
Epoch 

In [ ]:
# Evaluating the model
loss, accuracy = model.evaluate(X_test, y_test)

53/53 [==============================] - 0s 2ms/step - loss: 0.0103 - accuracy: 0.9880


In [ ]:
# New predictions (probabilities)
new_predictions = model.predict(X_test)
print(new_predictions)

53/53 [==============================] - 0s 2ms/step
[[9.9709564e-01]
 [1.9334954e-07]
 [1.4005596e-08]
 ...
 [2.9883932e-08]
 [9.9997580e-01]
 [3.8545064e-08]]


In [ ]:
# Defining a threshold
print(new_predictions > 0.5)

[[ True]
 [False]
 [False]
 ...
 [False]
 [ True]
 [False]]


In [ ]:
# Creating the confusion matrix
cm = confusion_matrix(y_pred=(new_predictions> 0.5), y_true=y_test)
print(cm)